In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier , DecisionTreeRegressor
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
df = pd.read_excel('ANZ synthesised transaction dataset.xlsx')

In [3]:
df.sample(5)

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
1240,authorized,1.0,NaN,ACC-3481401842,AUD,115.74 -31.72,POS,58b71a01-cda5-4d0a-bd8e-95dfd1213e89,NaN,Tiffany,...,25,Pearsall,WA,2018-08-10T14:38:57.000+0000,28.78,2686be814d244dfd9603e5e42cc01c1e,Australia,CUS-1669695324,115.83 -31.79,debit
11671,posted,NaN,NaN,ACC-90814749,AUD,151.68 -32.98,PAYMENT,NaN,NaN,Christopher,...,35,NaN,NaN,2018-10-28T20:00:00.000+0000,133.00,2fcc2d376bb848c2a9140e575bb2eab2,Australia,CUS-1896554896,NaN,debit
8744,authorized,1.0,NaN,ACC-2673069055,AUD,152.99 -27.49,SALES-POS,8fa7c1a9-28e3-4b02-92ef-58510a010425,NaN,Richard,...,24,Fortitude Valley,QLD,2018-10-06T16:29:18.000+0000,84.82,546257e7215c4402b5896e62d7ce7225,Australia,CUS-51506836,153.04 -27.46,debit
1162,authorized,1.0,NaN,ACC-2776252858,AUD,144.95 -37.76,SALES-POS,f113f8ba-fb0a-4517-b231-7cda3ec8b3fb,NaN,Kristin,...,43,Chirnside Park,VIC,2018-08-09T22:53:03.000+0000,22.02,adad7c7f42484a1c8a7cb0868f0d2537,Australia,CUS-4123612273,145.3 -37.77,debit
9830,authorized,1.0,NaN,ACC-1799207998,AUD,150.68 -33.79,POS,59ba0b94-0a95-4a8c-b11c-7ee934e3ea4d,NaN,Susan,...,20,West Perth,WA,2018-10-14T23:22:13.000+0000,24.77,a38962f6f7084a8c8ac9e151891aa504,Australia,CUS-164374203,115.84 -31.95,debit


In [4]:
df['txn_description'].unique()

array(['POS', 'SALES-POS', 'PAYMENT', 'INTER BANK', 'PAY/SALARY',
       'PHONE BANK'], dtype=object)

In [5]:
df_salary = df[df['txn_description']=='PAY/SALARY'].groupby('customer_id').mean()
df_salary

,card_present_flag,merchant_code,balance,age,amount
customer_id,,,,,
CUS-1005756958,NaN,0.0,4718.665385,53,970.47
CUS-1117979751,NaN,0.0,11957.202857,21,3578.65
CUS-1140341822,NaN,0.0,5841.720000,28,1916.51
CUS-1147642491,NaN,0.0,8813.467692,34,1711.39
CUS-1196156254,NaN,0.0,23845.717143,34,3903.73
...,...,...,...,...,...
CUS-72755508,NaN,0.0,4558.675833,35,725.32
CUS-809013380,NaN,0.0,5579.186154,21,1037.07
CUS-860700529,NaN,0.0,4221.468333,30,1808.62


In [6]:
#creating annual salary column
salary = []

for customer_id in df["customer_id"]:
    salary.append(int(df_salary.loc[customer_id]["amount"]))
    
df["annual_salary"] = salary

In [7]:
df_cus = df.groupby("customer_id").mean()
df_cus.head()

,card_present_flag,merchant_code,balance,age,amount,annual_salary
customer_id,,,,,,
CUS-1005756958,0.812500,0.0,2275.852055,53,222.862603,970
CUS-1117979751,0.826923,0.0,9829.929000,21,339.843700,3578
CUS-1140341822,0.815385,0.0,5699.212250,28,212.632500,1916
CUS-1147642491,0.750000,0.0,9032.841186,34,245.600169,1711
CUS-1196156254,0.785276,0.0,22272.433755,34,147.145796,3903


# Linear Regression

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X = df_cus.drop('annual_salary',axis=1)
y = df_cus['annual_salary']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [11]:
lin_reg = LinearRegression()

In [12]:
lin_reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
lin_predict = lin_reg.predict(X_test)

In [14]:
from sklearn import metrics

In [15]:
print('Mean Square error: ',metrics.mean_squared_error(y_test,lin_predict))     
print('Root Mean Square error: ',np.sqrt(metrics.mean_squared_log_error(y_test,lin_predict)))     

Mean Square error:  2905448.018394455
Root Mean Square error:  0.5586985428234099


# Decision Tree

In [16]:
df_dt = df[['txn_description','age','movement','gender']]

In [17]:
df_dt = pd.get_dummies(df_dt)
df_dt.head()

,age,txn_description_INTER BANK,txn_description_PAY/SALARY,txn_description_PAYMENT,txn_description_PHONE BANK,txn_description_POS,txn_description_SALES-POS,movement_credit,movement_debit,gender_F,gender_M
0,26,0,0,0,0,1,0,0,1,1,0
1,26,0,0,0,0,0,1,0,1,1,0
2,38,0,0,0,0,1,0,0,1,0,1
3,40,0,0,0,0,0,1,0,1,1,0
4,26,0,0,0,0,0,1,0,1,1,0


In [18]:
X = df_dt
y = df['annual_salary']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [25]:
dt_regress = DecisionTreeRegressor()

In [26]:
dt_regress.fit(X_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [27]:
dt_regress_predict = dt_regress.predict(X_test)

In [33]:
print('Mean Square error: ',metrics.mean_squared_error(y_test,dt_regress_predict))     
print('Root Mean Square error: ',np.sqrt(metrics.mean_squared_log_error(y_test,dt_regress_predict)))

Mean Square error:  782750.4864514983
Root Mean Square error:  0.37918003612688783
